<a href="https://colab.research.google.com/github/bramhini88/FMML-LABS.ipynb/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging the validation accuracy across multiple splits (e.g., in k-fold cross-validation) generally provides more consistent and reliable results compared to using a single split. Here's why:

1. Reduces Variance Due to Splitting

A single train-validation split can produce results that are highly dependent on how the data is divided. For example, if the split is unbalanced (e.g., contains more difficult or easy examples in one subset), the validation accuracy might not represent the model's true performance.

By averaging across multiple splits, the effect of any single "lucky" or "unlucky" split is diminished, leading to a more stable and reliable estimate.


2. Accounts for Data Distribution

Cross-validation ensures the model is trained and validated on different portions of the data. This accounts for variability in the data distribution, making the evaluation more robust.

For example, if the dataset has rare or underrepresented classes, averaging over multiple splits helps to mitigate bias introduced by their uneven distribution in a single split.


3. Better Generalization Estimation

The averaged performance over multiple splits is a better approximation of how well the model is likely to perform on unseen data. It smooths out the noise caused by idiosyncrasies in specific splits.


Example: K-Fold Cross-Validation

In k-fold cross-validation, the dataset is divided into  subsets (or folds). The model is trained  times, each time using  folds for training and the remaining fold for validation.



2. Does it give more accurate estimate of test accuracy?
Yes, averaging validation accuracy across multiple splits (e.g., in k-fold cross-validation) generally provides a more accurate estimate of test accuracy compared to using a single train-validation split. Here's why:

1. Better Representation of Data

In a single split, some data points may not be representative of the overall distribution, leading to biased estimates. Cross-validation ensures that every data point is used for both training and validation across multiple runs.

This provides a more comprehensive evaluation of how the model will perform on unseen data, which improves the reliability of the estimate for test accuracy.


2. Reduction in Variance

The performance of a model can vary significantly depending on how the data is split. Averaging over multiple splits reduces this variance, yielding a more stable and realistic estimate of the test accuracy.


3. Minimization of Sampling Bias

A single train-test split might accidentally favor the model if the test set contains easier examples or disadvantage it if the test set contains outliers or harder examples. Cross-validation distributes these biases across folds, reducing their impact on the accuracy estimate.


4. Generalization Capability

Since the model is trained and validated on different subsets of the data in each fold, it simulates multiple test scenarios. The averaged validation accuracy reflects the model’s ability to generalize across various subsets of the dataset, making it closer to the true test accuracy.


5. More Robust for Small Datasets

For small datasets, using a single train-test split might leave the test set too small or



3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The number of iterations (e.g., the number of folds in k-fold cross-validation or the number of random splits in Monte Carlo cross-validation) directly affects the reliability and accuracy of the estimate. Here's how it influences the estimate:


---

1. Improved Reliability with More Iterations

Variance Reduction: As the number of iterations increases, the impact of individual splits diminishes. The average validation accuracy becomes more stable and less sensitive to specific splits, leading to a more reliable estimate.

Convergence to True Mean: With more iterations, the estimate better approximates the true underlying performance of the model because it considers more variations in the data splits.



---

2. Diminishing Returns Beyond a Certain Point

After a sufficient number of iterations, the accuracy estimate converges, and additional iterations contribute little improvement. This is due to the law of diminishing returns, where the benefit of adding more iterations decreases as the estimate stabilizes.

For small datasets, very high numbers of iterations might also lead to overfitting or unnecessary computational expense without significantly improving the accuracy estimate.



---

3. Trade-Off: Bias vs. Variance

Small Number of Iterations: With fewer iterations, the estimate may have higher variance (sensitive to the specific splits) but lower bias (because each split uses more training data in k-fold cross-validation).

Large Number of Iterations: More iterations reduce variance, but in methods like k-fold cross-validation, the bias might increase slightly because smaller training sets are used in each fold.



---

4. Computational Cost

Higher iterations require retraining the model multiple times, which can become computationally expensive, especially for large datasets or complex models. Balancing computational cost with estimation reliability is crucial.



---

5. Practical Considerations

k-Fold Cross-Validation: Common choices like  or  are often sufficient to balance reliability and computational cost. Increasing  beyond 10 rarely provides significant improvement in the accuracy estimate.

Monte Carlo Cross-Validation: Using 50–100 random splits typically provides a good balance between accuracy and computational cost, but more iterations may be needed if the dataset is small or highly variable.



---

Conclusion

Increasing the number of iterations generally improves the accuracy of the estimate by reducing variance and making the result more consistent. However, beyond a certain point, the improvements are minimal, and the added computational cost may not be justified. It’s essential to find a balance based on the dataset size, model complexity, and computational resources.


4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Increasing the number of iterations (or splits) in cross-validation can help mitigate some issues when working with very small training or validation datasets, but it is not a complete solution. Here’s how increasing iterations might help and where it falls short:

How Increasing Iterations Can Help

1. Better Use of Limited Data

With small datasets, each individual split might not be representative of the overall data distribution. By increasing the number of iterations (e.g., using more folds in k-fold cross-validation or more random splits in Monte Carlo cross-validation), you ensure that each data point gets a chance to appear in both training and validation sets across multiple iterations. This provides a better estimate of model performance by averaging over different subsets.

In k-fold cross-validation, each data point is used for validation exactly once, and this is repeated for every fold. More iterations can ensure that the model is evaluated in different ways, reducing the risk of bias due to the choice of training/validation sets.



2. Reduces Variance in Results

When using a small dataset, a single train-test split might lead to high variance in results. A small validation set might not be representative of the overall data, or the model might perform particularly well or poorly on it due to its size. By increasing the number of iterations, you get a more reliable average of performance metrics, which reduces the impact of any outliers or unusual data points.



3. Exposure to More Diverse Data

More iterations can expose the model to more combinations of the data, giving the model a better understanding of the underlying patterns and reducing the chances of overfitting to specific subsets of data.




Limitations of Increasing Iterations

1. Not Solving the Fundamental Issue of Data Size

While more iterations can help make better use of a small dataset, it doesn’t fundamentally address the fact that the dataset is small. Even with repeated training and validation, a small dataset may still lead to overfitting or poor generalization, as there simply isn’t enough data for the model to learn from.

The model might still struggle to learn meaningful patterns from the small dataset, especially if the data is noisy or highly variable.



2. Increasing Computational Costs

More iterations mean more model training. With very small datasets, the computational overhead may not be significant, but for larger models or datasets, this can become a serious bottleneck. Each iteration requires training the model and performing validation, which can be time-consuming.



3. Risk of Overfitting to Validation Data

In cases where hyperparameter tuning is done using the validation results from each iteration, you risk overfitting the model to the validation data. While this does not directly affect the estimate of performance, it can lead to overly optimistic performance on the validation set, which might not generalize well to unseen data.



4. Effectiveness Diminishes with Extremely Small Datasets

If the dataset is extremely small (e.g., less than 50 samples), no matter how many iterations you perform, the model may still fail to generalize properly. In such cases, more sophisticated techniques such as data augmentation, transfer learning, or using pre-trained models may be more effective.




Alternative Approaches

For very small datasets, increasing iterations alone may not be sufficient. Here are some other strategies to improve model performance:

1. Data Augmentation

For datasets like images or text, data augmentation can artificially increase the dataset size by generating new data points through transformations (e.g., rotating images or altering text).



2. Transfer Learning

If you have access to a pre-trained model (especially for complex tasks like image classification or NLP), transfer learning can be a good solution. This involves using a model that was pre-trained on a large dataset and fine-tuning it on your smaller dataset.



3. Bootstrapping

Bootstrapping (resampling with

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.